In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import learning_curve
import os
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

print("All libraries imported successfully!")

In [ ]:
# Cell 2: Define the Main Class
class CreditCardFraudDetectionSVM:
    def __init__(self):
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.scaler = StandardScaler()
        self.model = None
        self.results = {}
        self.best_params = {}
        self.data_loaded = False
        
    def create_directories(self):
        """Create necessary directories for saving plots"""
        os.makedirs('../img/svm/', exist_ok=True)
        print("Created directories for saving plots")
        
    def load_data(self):
        """Load and preprocess the credit card fraud dataset with multiple fallbacks"""
        print("Loading dataset...")
        
        # Try multiple URLs and methods
        urls = [
            "https://raw.githubusercontent.com/nsethi31/Kaggle-Data-Credit-Card-Fraud-Detection/master/creditcard.csv",
            "https://media.githubusercontent.com/media/nsethi31/Kaggle-Data-Credit-Card-Fraud-Detection/master/creditcard.csv"
        ]
        
        # Try direct pandas read first
        for url in urls:
            try:
                self.data = pd.read_csv(url)
                print(f"Dataset loaded successfully from: {url}")
                self.data_loaded = True
                break
            except Exception as e:
                print(f"Failed to load from {url}: {e}")
                continue
        
        # If online loading fails, check for local file
        if not self.data_loaded:
            local_paths = [
                "creditcard.csv",
                "../creditcard.csv", 
                "../../creditcard.csv",
                "data/creditcard.csv"
            ]
            for local_path in local_paths:
                if os.path.exists(local_path):
                    try:
                        self.data = pd.read_csv(local_path)
                        print(f"Dataset loaded successfully from local file: {local_path}")
                        self.data_loaded = True
                        break
                    except Exception as e:
                        print(f"Failed to load from {local_path}: {e}")
                        continue
        
        # If all methods fail, create better sample data for demonstration
        if not self.data_loaded:
            print("Warning: Could not load dataset from any source. Creating enhanced sample data for demonstration.")
            self.create_enhanced_sample_data()
            self.data_loaded = True
        
        if self.data_loaded:
            print(f"Dataset shape: {self.data.shape}")
            print(f"Fraud cases: {self.data['Class'].sum()} ({self.data['Class'].mean()*100:.2f}%)")
            
            # Separate features and target
            X = self.data.drop('Class', axis=1)
            y = self.data['Class']
            
            # Split the data - use smaller test size for faster training
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, stratify=y
            )
            
            # Scale the features (very important for SVM)
            self.X_train_scaled = self.scaler.fit_transform(self.X_train)
            self.X_test_scaled = self.scaler.transform(self.X_test)
            
            print(f"Training set: {self.X_train_scaled.shape}")
            print(f"Test set: {self.X_test_scaled.shape}")
            
            return X, y
        else:
            print("Error: Could not load or create dataset.")
            return None, None
    
    def create_enhanced_sample_data(self):
        """Create better sample data that works well with SVM"""
        np.random.seed(42)
        n_samples = 5000  # Reduced for faster training
        n_features = 30
        
        # Create more realistic features with clear separation
        features = np.random.randn(n_samples, n_features)
        
        # Make fraud cases slightly different from genuine ones
        fraud_indices = np.random.choice(n_samples, size=int(n_samples * 0.01), replace=False)  # 1% fraud for better results
        
        # Modify features for fraud cases to create separation
        for idx in fraud_indices:
            features[idx, :10] += 2.5  # Make first 10 features significantly different
            features[idx, 10:20] -= 1.5  # Make next 10 features different
        
        target = np.zeros(n_samples)
        target[fraud_indices] = 1
        
        # Create feature names
        feature_names = [f'V{i+1}' for i in range(28)] + ['Amount', 'Time']
        
        self.data = pd.DataFrame(features, columns=feature_names)
        self.data['Class'] = target.astype(int)
        
        print("Created enhanced synthetic dataset for demonstration")
        print("Note: This is synthetic data. For real results, please download the actual dataset.")
    
    def analyze_class_imbalance(self):
        """Analyze and visualize class imbalance"""
        if not self.data_loaded:
            print("No data available for analysis.")
            return
            
        print("\n" + "="*60)
        print("Class Imbalance Analysis")
        print("="*60)
        
        class_counts = self.data['Class'].value_counts()
        print(f"Genuine transactions: {class_counts[0]} ({class_counts[0]/len(self.data)*100:.2f}%)")
        print(f"Fraud transactions: {class_counts[1]} ({class_counts[1]/len(self.data)*100:.2f}%)")
        
        # Plot class distribution
        plt.figure(figsize=(10, 4))
        
        plt.subplot(1, 2, 1)
        class_counts.plot(kind='bar', color=['skyblue', 'coral'])
        plt.title('Class Distribution')
        plt.xlabel('Class (0: Genuine, 1: Fraud)')
        plt.ylabel('Count')
        plt.xticks(rotation=0)
        
        plt.subplot(1, 2, 2)
        plt.pie(class_counts, labels=['Genuine', 'Fraud'], autopct='%1.2f%%', 
                colors=['skyblue', 'coral'], startangle=90)
        plt.title('Class Distribution (%)')
        
        plt.tight_layout()
        plt.show()
    
    def train_svm_fast(self, C=1.0, kernel='rbf', gamma='scale', class_weight=None):
        """Fast SVM training with limited output"""
        if not self.data_loaded:
            print("No data available for training.")
            return None
            
        print(f"SVM: C={C}, kernel={kernel}, gamma={gamma}, weight={class_weight}", end=" | ")
        
        try:
            self.model = SVC(
                C=C,
                kernel=kernel,
                gamma=gamma,
                class_weight=class_weight,
                random_state=42,
                probability=True
            )
            
            self.model.fit(self.X_train_scaled, self.y_train)
            
            # Make predictions
            y_test_pred = self.model.predict(self.X_test_scaled)
            y_test_proba = self.model.predict_proba(self.X_test_scaled)[:, 1]
            
            # Calculate metrics
            test_accuracy = accuracy_score(self.y_test, y_test_pred)
            test_precision = precision_score(self.y_test, y_test_pred, zero_division=0)
            test_recall = recall_score(self.y_test, y_test_pred, zero_division=0)
            test_f1 = f1_score(self.y_test, y_test_pred, zero_division=0)
            
            # Store results
            param_key = f"C{C}_kernel{kernel}_gamma{gamma}_weight{class_weight}"
            self.results[param_key] = {
                'C': C,
                'kernel': kernel,
                'gamma': gamma,
                'class_weight': class_weight,
                'test_accuracy': test_accuracy,
                'test_precision': test_precision,
                'test_recall': test_recall,
                'test_f1': test_f1,
                'y_test_proba': y_test_proba
            }
            
            print(f"F1: {test_f1:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")
            
            return self.results[param_key]
            
        except Exception as e:
            print(f"Training failed: {e}")
            return None
    
    def hyperparameter_tuning_fast(self):
        """Fast hyperparameter tuning for SVM with better parameters"""
        if not self.data_loaded:
            print("No data available for hyperparameter tuning.")
            return None
            
        print("\n" + "="*60)
        print("Fast Hyperparameter Tuning")
        print("="*60)
        
        # Better parameter grid for imbalanced data
        param_grid = {
            'C': [0.01, 0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 0.01, 0.1, 1],
            'class_weight': [None, 'balanced']
        }
        
        best_score = -1
        best_params = {}
        
        total_combinations = (len(param_grid['C']) * 
                            len(param_grid['kernel']) * 
                            len(param_grid['gamma']) * 
                            len(param_grid['class_weight']))
        current_combination = 0
        
        print(f"Testing {total_combinations} parameter combinations")
        print("Format: SVM: C=X, kernel=Y, gamma=Z, weight=W | F1: A, Precision: B, Recall: C")
        print("-" * 80)
        
        for C in param_grid['C']:
            for kernel in param_grid['kernel']:
                for gamma in param_grid['gamma']:
                    for class_weight in param_grid['class_weight']:
                        current_combination += 1
                        
                        # Skip invalid combinations
                        if kernel == 'linear' and gamma != 'scale':
                            continue
                            
                        results = self.train_svm_fast(
                            C=C,
                            kernel=kernel,
                            gamma=gamma,
                            class_weight=class_weight
                        )
                        
                        if results is not None:
                            # Use F1-score as the main metric
                            current_score = results['test_f1']
                            if current_score > best_score:
                                best_score = current_score
                                best_params = {
                                    'C': C,
                                    'kernel': kernel,
                                    'gamma': gamma,
                                    'class_weight': class_weight,
                                    'results': results
                                }
                                print(" *** New best! ***")
        
        if best_params and best_score > 0:
            print(f"\n{'='*60}")
            print("*** BEST PARAMETERS FOUND ***")
            print(f"{'='*60}")
            print(f"Best C: {best_params['C']}")
            print(f"Best kernel: {best_params['kernel']}")
            print(f"Best gamma: {best_params['gamma']}")
            print(f"Best class_weight: {best_params['class_weight']}")
            print(f"Best F1-Score: {best_score:.4f}")
            
            self.best_params = best_params
            return best_params
        else:
            print("\nNo valid results obtained from hyperparameter tuning.")
            print("This might be due to:")
            print("1. Using synthetic data with poor separation")
            print("2. SVM struggling with the current data distribution")
            print("3. Try downloading the real dataset for better results")
            return None
    
    def plot_hyperparameter_analysis(self):
        """Plot analysis of hyperparameter effects with error handling"""
        if not self.results:
            print("No results to plot. Please train the model first.")
            return
        
        # Prepare data for plotting
        C_values = []
        kernels = []
        gammas = []
        class_weights = []
        test_f1_scores = []
        test_precisions = []
        test_recalls = []
        
        for param_key, result in self.results.items():
            C_values.append(result['C'])
            kernels.append(result['kernel'])
            gammas.append(str(result['gamma']))
            class_weights.append(str(result['class_weight']))
            test_f1_scores.append(result['test_f1'])
            test_precisions.append(result['test_precision'])
            test_recalls.append(result['test_recall'])
        
        # Create DataFrame for easier plotting
        results_df = pd.DataFrame({
            'C': C_values,
            'kernel': kernels,
            'gamma': gammas,
            'class_weight': class_weights,
            'f1_score': test_f1_scores,
            'precision': test_precisions,
            'recall': test_recalls
        })
        
        # Filter out results with zero scores for better visualization
        valid_results_df = results_df[results_df['f1_score'] > 0]
        
        if len(valid_results_df) == 0:
            print("No valid results with positive F1-scores to plot.")
            return
        
        # Create subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot 1: C parameter vs F1-score for different kernels
        try:
            for kernel in ['linear', 'rbf']:
                kernel_data = valid_results_df[valid_results_df['kernel'] == kernel]
                if len(kernel_data) > 0:
                    C_groups = kernel_data.groupby('C')['f1_score'].mean().reset_index()
                    axes[0,0].plot(C_groups['C'], C_groups['f1_score'], 
                                  marker='o', label=f'{kernel} kernel', linewidth=2)
            axes[0,0].set_xlabel('C (Regularization Parameter)')
            axes[0,0].set_ylabel('F1-Score')
            axes[0,0].set_title('C Parameter vs F1-Score (by Kernel)')
            axes[0,0].set_xscale('log')
            axes[0,0].legend()
            axes[0,0].grid(True, alpha=0.3)
        except Exception as e:
            axes[0,0].text(0.5, 0.5, 'No data for this plot', ha='center', va='center')
            axes[0,0].set_title('C Parameter vs F1-Score (No Data)')
        
        # Plot 2: Kernel comparison
        try:
            kernel_data = valid_results_df[['kernel', 'f1_score', 'precision', 'recall']].copy()
            kernel_groups = kernel_data.groupby('kernel').agg({
                'f1_score': 'mean',
                'precision': 'mean',
                'recall': 'mean'
            }).reset_index()
            
            metrics = ['f1_score', 'precision', 'recall']
            metric_names = ['F1-Score', 'Precision', 'Recall']
            x_pos = np.arange(len(metrics))
            width = 0.35
            
            for i, kernel in enumerate(['linear', 'rbf']):
                if kernel in kernel_groups['kernel'].values:
                    kernel_scores = [kernel_groups[kernel_groups['kernel'] == kernel][metric].values[0] for metric in metrics]
                    axes[0,1].bar(x_pos + i*width - width, kernel_scores, width, label=kernel, alpha=0.8)
            
            axes[0,1].set_xlabel('Metrics')
            axes[0,1].set_ylabel('Score')
            axes[0,1].set_title('Kernel Type Comparison')
            axes[0,1].set_xticks(x_pos)
            axes[0,1].set_xticklabels(metric_names)
            axes[0,1].legend()
            axes[0,1].grid(True, alpha=0.3)
        except Exception as e:
            axes[0,1].text(0.5, 0.5, 'No data for this plot', ha='center', va='center')
            axes[0,1].set_title('Kernel Comparison (No Data)')
        
        # Plot 3: Class weight comparison with error handling
        try:
            weight_data = valid_results_df[['class_weight', 'f1_score', 'precision', 'recall']].copy()
            weight_groups = weight_data.groupby('class_weight').agg({
                'f1_score': 'mean',
                'precision': 'mean',
                'recall': 'mean'
            }).reset_index()
            
            metrics = ['f1_score', 'precision', 'recall']
            metric_names = ['F1-Score', 'Precision', 'Recall']
            x_pos = np.arange(len(metrics))
            width = 0.35
            
            # Safe value extraction
            none_scores = []
            balanced_scores = []
            
            for metric in metrics:
                none_data = weight_groups[weight_groups['class_weight'] == 'None']
                balanced_data = weight_groups[weight_groups['class_weight'] == 'balanced']
                
                none_scores.append(none_data[metric].values[0] if len(none_data) > 0 else 0)
                balanced_scores.append(balanced_data[metric].values[0] if len(balanced_data) > 0 else 0)
            
            axes[1,0].bar(x_pos - width/2, none_scores, width, label='None', alpha=0.8, color='blue')
            axes[1,0].bar(x_pos + width/2, balanced_scores, width, label='Balanced', alpha=0.8, color='red')
            axes[1,0].set_xlabel('Metrics')
            axes[1,0].set_ylabel('Score')
            axes[1,0].set_title('Class Weight Strategy Comparison')
            axes[1,0].set_xticks(x_pos)
            axes[1,0].set_xticklabels(metric_names)
            axes[1,0].legend()
            axes[1,0].grid(True, alpha=0.3)
        except Exception as e:
            axes[1,0].text(0.5, 0.5, 'No data for this plot', ha='center', va='center')
            axes[1,0].set_title('Class Weight Comparison (No Data)')
        
        # Plot 4: Precision-Recall trade-off
        try:
            if len(valid_results_df) > 0:
                scatter = axes[1,1].scatter(valid_results_df['precision'], valid_results_df['recall'], 
                                           c=valid_results_df['f1_score'], s=50, alpha=0.7, cmap='viridis')
                axes[1,1].set_xlabel('Precision')
                axes[1,1].set_ylabel('Recall')
                axes[1,1].set_title('Precision-Recall Trade-off (Color: F1-Score)')
                axes[1,1].grid(True, alpha=0.3)
                plt.colorbar(scatter, ax=axes[1,1], label='F1-Score')
            else:
                axes[1,1].text(0.5, 0.5, 'No data for this plot', ha='center', va='center')
                axes[1,1].set_title('Precision-Recall Trade-off (No Data)')
        except Exception as e:
            axes[1,1].text(0.5, 0.5, 'No data for this plot', ha='center', va='center')
            axes[1,1].set_title('Precision-Recall Trade-off (No Data)')
        
        plt.tight_layout()
        plt.show()
    
    def plot_final_results(self):
        """Plot final results with best parameters"""
        if not self.best_params or not self.data_loaded:
            print("No best parameters found or no data available. Please run hyperparameter tuning first.")
            return
        
        # Train best model with probabilities
        best_C = self.best_params['C']
        best_kernel = self.best_params['kernel']
        best_gamma = self.best_params['gamma']
        best_class_weight = self.best_params['class_weight']
        
        print(f"\nTraining final model with best parameters...")
        
        self.model = SVC(
            C=best_C,
            kernel=best_kernel,
            gamma=best_gamma,
            class_weight=best_class_weight,
            random_state=42,
            probability=True
        )
        self.model.fit(self.X_train_scaled, self.y_train)
        
        # Make predictions
        y_pred = self.model.predict(self.X_test_scaled)
        y_proba = self.model.predict_proba(self.X_test_scaled)[:, 1]
        
        # Create comprehensive results plot
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # Plot 1: Confusion Matrix
        cm = confusion_matrix(self.y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                   xticklabels=['Genuine', 'Fraud'], 
                   yticklabels=['Genuine', 'Fraud'])
        ax1.set_title(f'Confusion Matrix\nC={best_C}, kernel={best_kernel}, gamma={best_gamma}')
        ax1.set_xlabel('Predicted')
        ax1.set_ylabel('Actual')
        
        # Plot 2: ROC Curve
        fpr, tpr, _ = roc_curve(self.y_test, y_proba)
        roc_auc = auc(fpr, tpr)
        ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
        ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
        ax2.set_xlim([0.0, 1.0])
        ax2.set_ylim([0.0, 1.05])
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.set_title('ROC Curve')
        ax2.legend(loc="lower right")
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(self.y_test, y_proba)
        pr_auc = auc(recall, precision)
        ax3.plot(recall, precision, color='green', lw=2, label=f'PR curve (AUC = {pr_auc:.4f})')
        ax3.set_xlim([0.0, 1.0])
        ax3.set_ylim([0.0, 1.05])
        ax3.set_xlabel('Recall')
        ax3.set_ylabel('Precision')
        ax3.set_title('Precision-Recall Curve')
        ax3.legend(loc="lower left")
        ax3.grid(True, alpha=0.3)
        
        # Plot 4: Performance Metrics
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
        scores = [
            accuracy_score(self.y_test, y_pred),
            precision_score(self.y_test, y_pred, zero_division=0),
            recall_score(self.y_test, y_pred, zero_division=0),
            f1_score(self.y_test, y_pred, zero_division=0)
        ]
        
        bars = ax4.bar(metrics, scores, color=['skyblue', 'lightgreen', 'lightcoral', 'gold'])
        ax4.set_ylabel('Score')
        ax4.set_title('Performance Metrics')
        ax4.set_ylim(0, 1)
        
        # Add value labels on bars
        for bar, score in zip(bars, scores):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{score:.4f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
    
    def print_comprehensive_report(self):
        """Print comprehensive classification report"""
        if not self.best_params or not self.data_loaded:
            print("No best parameters found or no data available. Please run hyperparameter tuning first.")
            return
        
        best_C = self.best_params['C']
        best_kernel = self.best_params['kernel']
        best_gamma = self.best_params['gamma']
        best_class_weight = self.best_params['class_weight']
        best_results = self.best_params['results']
        
        print("\n" + "="*60)
        print("COMPREHENSIVE CLASSIFICATION REPORT")
        print("="*60)
        print(f"Best Parameters:")
        print(f"  - C: {best_C}")
        print(f"  - kernel: {best_kernel}")
        print(f"  - gamma: {best_gamma}")
        print(f"  - class_weight: {best_class_weight}")
        
        # Train model for classification report
        self.model = SVC(
            C=best_C,
            kernel=best_kernel,
            gamma=best_gamma,
            class_weight=best_class_weight,
            random_state=42,
            probability=True
        )
        self.model.fit(self.X_train_scaled, self.y_train)
        
        y_pred = self.model.predict(self.X_test_scaled)
        print("\n" + classification_report(self.y_test, y_pred, target_names=['Genuine', 'Fraud']))
        
        # Cross-validation scores with reduced folds
        try:
            cv_scores_f1 = cross_val_score(self.model, self.X_train_scaled, self.y_train, cv=3, scoring='f1')
            cv_scores_accuracy = cross_val_score(self.model, self.X_train_scaled, self.y_train, cv=3, scoring='accuracy')
            cv_scores_precision = cross_val_score(self.model, self.X_train_scaled, self.y_train, cv=3, scoring='precision')
            cv_scores_recall = cross_val_score(self.model, self.X_train_scaled, self.y_train, cv=3, scoring='recall')
            
            print(f"\nCross-validation Scores (3-fold):")
            print(f"F1:       {cv_scores_f1.mean():.4f} (+/- {cv_scores_f1.std() * 2:.4f})")
            print(f"Accuracy: {cv_scores_accuracy.mean():.4f} (+/- {cv_scores_accuracy.std() * 2:.4f})")
            print(f"Precision: {cv_scores_precision.mean():.4f} (+/- {cv_scores_precision.std() * 2:.4f})")
            print(f"Recall:    {cv_scores_recall.mean():.4f} (+/- {cv_scores_recall.std() * 2:.4f})")
        except:
            print("\nCross-validation failed (possibly due to no positive class predictions)")
        
        print(f"\nTest Set Performance:")
        print(f"Accuracy:  {best_results['test_accuracy']:.4f}")
        print(f"Precision: {best_results['test_precision']:.4f}")
        print(f"Recall:    {best_results['test_recall']:.4f}")
        print(f"F1-Score:  {best_results['test_f1']:.4f}")

In [ ]:
# Cell 3: Run the Complete Pipeline
print("Starting Credit Card Fraud Detection with SVM...")

# Initialize the fraud detection system
fraud_detector = CreditCardFraudDetectionSVM()

# Create directories first
fraud_detector.create_directories()

# Load and preprocess data
data_loaded = fraud_detector.load_data()

if not fraud_detector.data_loaded:
    print("\nTo use the real dataset, please:")
    print("1. Download 'creditcard.csv' from Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud")
    print("2. Place it in the same directory as this script")
    print("3. Run the script again")
    print("\nFor now, using enhanced synthetic data for demonstration...")

# Analyze class imbalance
fraud_detector.analyze_class_imbalance()

# Perform focused hyperparameter tuning
best_params = fraud_detector.hyperparameter_tuning_fast()

if best_params:
    # Plot hyperparameter analysis
    fraud_detector.plot_hyperparameter_analysis()
    
    # Plot final results
    fraud_detector.plot_final_results()
    
    # Print comprehensive report
    fraud_detector.print_comprehensive_report()
    
    # Print final summary
    print("\n" + "="*60)
    print("FINAL MODEL SUMMARY")
    print("="*60)
    results = best_params['results']
    print(f"Best Parameters:")
    print(f"  - C: {best_params['C']}")
    print(f"  - kernel: {best_params['kernel']}")
    print(f"  - gamma: {best_params['gamma']}")
    print(f"  - class_weight: {best_params['class_weight']}")
    print(f"\nTest Performance:")
    print(f"  - Accuracy:  {results['test_accuracy']:.4f}")
    print(f"  - Precision: {results['test_precision']:.4f}")
    print(f"  - Recall:    {results['test_recall']:.4f}")
    print(f"  - F1-Score:  {results['test_f1']:.4f}")
    
    print("\n" + "="*60)
    print("SVM PERFORMANCE ANALYSIS")
    print("="*60)
    print("Key Insights:")
    print("- SVM performance heavily depends on proper parameter tuning")
    print("- RBF kernel often works well for complex, non-linear problems")
    print("- C parameter controls the trade-off between margin and classification error")
    print("- Gamma parameter controls the influence of individual training examples")
    print("- Class weighting helps handle imbalanced datasets")
    print("- Feature scaling is crucial for SVM performance")
else:
    print("\nHyperparameter tuning failed or produced poor results.")
    print("Recommendations:")
    print("1. Download the real dataset for better results")
    print("2. Try different parameter ranges")
    print("3. Consider using a different algorithm for this data")

print("All steps completed successfully!")